In [5]:
import pandas as pd
data = pd.read_csv("online_shoppers_intention.csv")

In [25]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
# candidates for one-hot-encoding (OHE)
print(data.shape[1])
print(data.columns)

featuresOHN = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType']
#data[featuresOHN] = data[featuresOHN].astype(str)
transformer = make_column_transformer((OneHotEncoder(categories='auto', sparse_output=False), featuresOHN), remainder='passthrough')
transformed = transformer.fit_transform(data)
dataOHN = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
flist = dataOHN.columns.to_list()
newflist = [col.replace('remainder__', '') if col.startswith('remainder__') else col.replace('onehotencoder__', '') for col in flist]
dataOHN.columns = newflist

print(dataOHN.shape[1])
print(dataOHN.columns)

